<a href="https://colab.research.google.com/github/sweetbaer03/EN_Sentiment-Analysis/blob/main/friends_electra_%EA%B0%90%EC%A0%95%EC%82%AC%EC%A0%84%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FRIENDS Sentiment Analysis**


electra 모델 사용 

In [1]:
#transformers 설치(colab 사용시)
!pip install transformers

     |████████████████████████████████| 1.5MB 13.7MB/s 
     |████████████████████████████████| 2.9MB 53.4MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=59b0b55818286dde6cfbbc0e7ccca6ade1af96fb718ce10bd6ca2b8b84f8b678
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import time
import datetime
import json
import re
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

# 데이터처리

github에서 필요한 데이터파일 다운

In [3]:
!git clone https://github.com/sweetbaer03/EN_Sentiment-Analysis.git

Cloning into 'EN_Sentiment-Analysis'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 4), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [4]:
MAX_LEN = 85
batch_size = 32
epochs = 20

TOKEN_MODEL = 'google/electra-small-discriminator'
TRAIN_MODEL = 'google/electra-small-generator'

#TOKEN_MODEL = 'google/electra-base-discriminator'
#TRAIN_MODEL = 'google/electra-base-generator'

#TOKEN_MODEL = 'google/electra-large-discriminator'
#TRAIN_MODEL = 'google/electra-large-generator'

## 프랜즈

In [5]:
def jsonToDf(file_name):
  with open(file_name, encoding = 'utf-8', mode = 'r') as file:
    json_array = json.load(file)
  
  result = pd.DataFrame.from_dict(json_array[0])

  is_first = True
  for array in json_array:
    if is_first:
      is_first = False
      continue
    
    temp_df = pd.DataFrame.from_dict(array)
    result = result.append(temp_df, ignore_index = True)

  return result

In [6]:
train_data = jsonToDf('EN_Sentiment-Analysis/data_in/friends_train.json')
dev_data   = jsonToDf('EN_Sentiment-Analysis/data_in/friends_dev.json')
test_data  = jsonToDf('EN_Sentiment-Analysis/data_in/friends_test.json')

In [7]:
#합치기 위해 포멧 통일
train_data.drop(labels=['speaker','annotation'], axis="columns", inplace=True)
dev_data.drop(labels=['speaker','annotation'], axis="columns", inplace=True)
test_data.drop(labels=['speaker','annotation'], axis="columns", inplace=True)

In [8]:
emotions = train_data['emotion'].values

In [9]:
train_data

,utterance,emotion
0,also I was the point person on my companys tr...,neutral
1,You mustve had your hands full.,neutral
2,That I did. That I did.,neutral
3,So lets talk a little bit about your duties.,neutral
4,My duties? All right.,surprise
...,...,...
10556,You or me?,neutral
10557,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral
10558,"You guys are messing with me, right?",surprise
10559,Yeah.,neutral


In [10]:
dev_data

,utterance,emotion
0,"Oh my God, hes lost it. Hes totally lost it.",non-neutral
1,What?,surprise
2,"Or! Or, we could go to the bank, close our acc...",neutral
3,Youre a genius!,joy
4,"Aww, man, now we wont be bank buddies!",sadness
...,...,...
1173,No.,sadness
1174,What? Oh my God! Im gonna miss you so much!,sadness
1175,Im gonna miss you!,sadness
1176,I mean its the end of an era!,sadness


In [11]:
test_data

,utterance,emotion
0,Why do all youre coffee mugs have numbers on ...,surprise
1,Oh. Thats so Monica can keep track. That way ...,non-neutral
2,Y'know what?,neutral
3,It didnt.,neutral
4,"Okay, so what you used to have with Rachel, is...",joy
...,...,...
2759,"Hey, why dont you guys go get portraits done ...",neutral
2760,Thats a good idea! I bet they have one of tho...,joy
2761,"Yeah thats great! Next to that, Chandler won...",neutral
2762,Chandler what do you say?,neutral


## 캐글 테스트 데이터

In [12]:
test_df = pd.read_csv('EN_Sentiment-Analysis/data_in/en_data.csv')#, encoding = 'unicode_escape')

In [13]:
test_df

,id,i_dialog,i_utterance,speaker,utterance
0,0,0,0,Phoebe,"Alright, whadyou do with him?"
1,1,0,1,Monica,Oh! You're awake!
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...
3,3,0,3,Mr. Tribbiani,"Yeah, but this is"
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!
...,...,...,...,...,...
1618,1618,150,14,Joey,Nooo.
1619,1619,150,15,Lauren,"Hi, Kate!"
1620,1620,150,16,Kate,"Hi, Lauren."
1621,1621,150,17,Joey,"Hi, Lauren."


# 프랜즈 및 캐글테스트 마스크 처리

프랜즈 데이터 마스크

In [14]:
print('train_data : ', train_data.shape)
print(train_data.groupby(['emotion']).size().reset_index(name = 'count'))

train_data :  (10561, 2)
       emotion  count
0        anger    513
1      disgust    240
2         fear    185
3          joy   1283
4      neutral   4752
5  non-neutral   2017
6      sadness    351
7     surprise   1220


In [15]:
print('dev_data : ', dev_data.shape)
print(dev_data.groupby(['emotion']).size().reset_index(name = 'count'))

dev_data :  (1178, 2)
       emotion  count
0        anger     85
1      disgust     23
2         fear     29
3          joy    123
4      neutral    491
5  non-neutral    214
6      sadness     62
7     surprise    151


In [16]:
print('test_data : ', test_data.shape)
print(test_data.groupby(['emotion']).size().reset_index(name = 'count'))

test_data :  (2764, 2)
       emotion  count
0        anger    161
1      disgust     68
2         fear     32
3          joy    304
4      neutral   1287
5  non-neutral    541
6      sadness     85
7     surprise    286


In [17]:
print('test_df : ', test_df.shape)

test_df :  (1623, 5)


In [18]:
def cleaning1(str):
    replaceAll= str
    only_english = re.sub('\x92', '\'', replaceAll)
    return only_english

In [20]:
def getInputsAndLabels(dataset):
  data = dataset.copy(deep=True)

  utterances = data['utterance']
  utterances = ["[CLS] " + str(cleaning1(utterance)) + " [SEP]" for utterance in utterances]#이거다!
  
  encoder = LabelEncoder()
  labels = data['emotion'].values
  encoder.fit(labels)
  labels = encoder.transform(labels)

  tokenizer = ElectraTokenizer.from_pretrained(TOKEN_MODEL)
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, labels, attention_masks

In [21]:
def getInputsFromTest(dataset):
  data = dataset.copy(deep=True)

  utterances = data['utterance']
  utterances = ["[CLS] " + str(cleaning1(utterance)) + " [SEP]" for utterance in utterances]#이거다!
  
  tokenizer = ElectraTokenizer.from_pretrained(TOKEN_MODEL)
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

In [22]:
def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.id.tolist()
  return torch.tensor(input_index)

In [23]:
train_data

,utterance,emotion
0,also I was the point person on my companys tr...,neutral
1,You mustve had your hands full.,neutral
2,That I did. That I did.,neutral
3,So lets talk a little bit about your duties.,neutral
4,My duties? All right.,surprise
...,...,...
10556,You or me?,neutral
10557,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral
10558,"You guys are messing with me, right?",surprise
10559,Yeah.,neutral


In [24]:
train_inputs1, train_labels1, train_masks1 = getInputsAndLabels(train_data)
dev_inputs, dev_labels, dev_masks = getInputsAndLabels(dev_data)
test_inputs, test_labels, test_masks = getInputsAndLabels(test_data)
test_df_inputs, test_df_masks = getInputsFromTest(test_df)

In [25]:
print('전체 프랜즈 학  습 데이터의 개수: {}'.format(len(train_inputs1)))
print('전체 프랜즈 라  벨 데이터의 개수: {}'.format(len(train_labels1)))
print('전체 프랜즈 마스크 데이터의 개수: {}'.format(len(train_masks1)))

전체 프랜즈 학  습 데이터의 개수: 10561
전체 프랜즈 라  벨 데이터의 개수: 10561
전체 프랜즈 마스크 데이터의 개수: 10561


In [26]:
print('전체 프랜즈 dev학  습 데이터의 개수: {}'.format(len(dev_inputs)))
print('전체 프랜즈 dev라  벨 데이터의 개수: {}'.format(len(dev_labels)))
print('전체 프랜즈 dev마스크 데이터의 개수: {}'.format(len(dev_masks)))

전체 프랜즈 dev학  습 데이터의 개수: 1178
전체 프랜즈 dev라  벨 데이터의 개수: 1178
전체 프랜즈 dev마스크 데이터의 개수: 1178


In [27]:
print('전체 프랜즈 test학  습 데이터의 개수: {}'.format(len(test_inputs)))
print('전체 프랜즈 test라  벨 데이터의 개수: {}'.format(len(test_labels)))
print('전체 프랜즈 test마스크 데이터의 개수: {}'.format(len(test_masks)))

전체 프랜즈 test학  습 데이터의 개수: 2764
전체 프랜즈 test라  벨 데이터의 개수: 2764
전체 프랜즈 test마스크 데이터의 개수: 2764


In [28]:
print('전체 프랜즈 test1 학  습 데이터의 개수: {}'.format(len(test_df_inputs)))
print('전체 프랜즈 test1 마스크 데이터의 개수: {}'.format(len(test_df_masks)))

전체 프랜즈 test1 학  습 데이터의 개수: 1623
전체 프랜즈 test1 마스크 데이터의 개수: 1623


# IMDB데이터 사용

In [29]:
#IMDB data
train_data_db = pd.read_csv('EN_Sentiment-Analysis/data_in/labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)
train_data_db.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [30]:
def cleaning_mv(str):
    replaceAll= str
    only_english = re.sub('<br />', '', replaceAll)
    return only_english

In [31]:
# 감정을 숫자로 변환
def emotion_labeling_mv(emotion):
   return{0 : 'anger',1:'joy'}[emotion]

emotion_labels = []

for e in train_data_db['sentiment']:
   emotion_labels.append(emotion_labeling_mv(e))

train_data_db['label'] = emotion_labels
train_data_db[:10]

,id,sentiment,review,label
0,"""5814_8""",1,"""With all this stuff going down at the moment ...",joy
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...",joy
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...",anger
3,"""3630_4""",0,"""It must be assumed that those who praised thi...",anger
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...",joy
5,"""8196_8""",1,"""I dont know why people think this is such a b...",joy
6,"""7166_2""",0,"""This movie could have been very good, but com...",anger
7,"""10633_1""",0,"""I watched this video at a friend's house. I'm...",anger
8,"""319_1""",0,"""A friend of mine bought this film for £1, and...",anger
9,"""8713_10""",1,"""<br /><br />This movie is full of references....",joy


In [32]:
def getInputsAndLabels2(dataset):
  data = dataset.copy(deep=True)

  utterances = data['review']
  #utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  utterances = ["[CLS] " + str(cleaning_mv(utterance)) + " [SEP]" for utterance in utterances]
  
  encoder = LabelEncoder()
  labels = data['label'].values
  encoder.fit(labels)
  labels = encoder.transform(labels)

  tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, labels, attention_masks

In [33]:
train_inputs2, train_labels2, train_masks2 = getInputsAndLabels2(train_data_db)

In [34]:
print('전체 IMDB 학  습 데이터의 개수: {}'.format(len(train_inputs2)))
print('전체 IMDB 라  벨 데이터의 개수: {}'.format(len(train_labels2)))
print('전체 IMDB 마스크 데이터의 개수: {}'.format(len(train_masks2)))

전체 IMDB 학  습 데이터의 개수: 25000
전체 IMDB 라  벨 데이터의 개수: 25000
전체 IMDB 마스크 데이터의 개수: 25000


# 학습 데이터 합치기

In [35]:
train_masks0 = []
train_masks0.extend(train_masks1)
train_masks0.extend(train_masks2)
#train_masks0.extend(train_masks3)

In [36]:
train_labels0 = []
train_labels0.extend(train_labels1)
train_labels0.extend(train_labels2)
#train_labels0.extend(train_labels3)

In [37]:
train_inputs0 = []
train_inputs0.extend(train_inputs1)
train_inputs0.extend(train_inputs2)
#train_inputs0.extend(train_inputs3)

In [38]:
print('전체 학  습 데이터의 개수: {}'.format(len(train_inputs0)))
print('전체 라  벨 데이터의 개수: {}'.format(len(train_labels0)))
print('전체 마스크 데이터의 개수: {}'.format(len(train_masks0)))

전체 학  습 데이터의 개수: 35561
전체 라  벨 데이터의 개수: 35561
전체 마스크 데이터의 개수: 35561


In [39]:
train_inputs = torch.tensor(train_inputs0)
train_labels = torch.tensor(train_labels0)
train_masks = torch.tensor(train_masks0)

dev_inputs = torch.tensor(dev_inputs)
dev_labels = torch.tensor(dev_labels)
dev_masks = torch.tensor(dev_masks)

test_inputs = torch.tensor(test_inputs)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

test_df_index = getIndex(test_df)
test_df_inputs = torch.tensor(test_df_inputs)
test_df_masks = torch.tensor(test_df_masks)

In [40]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

test_df_data = TensorDataset(test_df_index, test_df_inputs, test_df_masks)
test_df_sampler = RandomSampler(test_df_data)
test_df_dataloader = DataLoader(test_df_data, sampler=test_df_sampler, batch_size=batch_size)

# **모델 구현**

In [41]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [42]:
model = ElectraForSequenceClassification.from_pretrained(TRAIN_MODEL, num_labels=8)
model.cuda()

Some weights of the model checkpoint at google/electra-small-generator were not used when initializing ElectraForSequenceClassification: ['generator_predictions.LayerNorm.weight', 'generator_predictions.LayerNorm.bias', 'generator_predictions.dense.weight', 'generator_predictions.dense.bias', 'generator_lm_head.weight', 'generator_lm_head.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-generator and are newly initializ

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

In [43]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )

total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# **학습**

In [44]:
from sklearn.metrics import f1_score

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [45]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
             
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()


        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in dev_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
     
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...
  Batch   500  of  1,112.    Elapsed: 0:00:35.
  Batch 1,000  of  1,112.    Elapsed: 0:01:11.

  Average training loss: 0.88
  Training epcoh took: 0:01:18

Running Validation...
  Accuracy: 0.47369
  Validation took: 0:00:00

======== Epoch 2 / 20 ========
Training...
  Batch   500  of  1,112.    Elapsed: 0:00:35.
  Batch 1,000  of  1,112.    Elapsed: 0:01:10.

  Average training loss: 0.66
  Training epcoh took: 0:01:17

Running Validation...
  Accuracy: 0.54165
  Validation took: 0:00:00

======== Epoch 3 / 20 ========
Training...
  Batch   500  of  1,112.    Elapsed: 0:00:35.
  Batch 1,000  of  1,112.    Elapsed: 0:01:10.

  Average training loss: 0.59
  Training epcoh took: 0:01:18

Running Validation...
  Accuracy: 0.55347
  Validation took: 0:00:00

======== Epoch 4 / 20 ========
Training...
  Batch   500  of  1,112.    Elapsed: 0:00:35.
  Batch 1,000  of  1,112.    Elapsed: 0:01:09.

  Average training loss: 0.54
  Training epcoh took

In [46]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


# f1-score parameter
from sklearn.metrics import f1_score
f1_score_avg = []
trues = []
preds = []

In [47]:
tmp_test_dataloader = DataLoader(test_df_data, sampler=test_df_sampler, batch_size=1)
test_result = test_df.copy(deep = True)
test_result = test_result.drop(columns = ['i_dialog', 'i_utterance', 'speaker'])
test_result['Predicted'] = 'default'
classes = [0,1,2,3,4,5,6,7]

encoder = LabelEncoder()
classes = emotions
encoder.fit(classes)
classes = encoder.transform(classes)

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(tmp_test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(tmp_test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = encoder.classes_[np.argmax(logits)]
    

    nb_eval_steps += 1

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  Batch   100  of  1,623.    Elapsed: 0:00:01.
  Batch   200  of  1,623.    Elapsed: 0:00:02.
  Batch   300  of  1,623.    Elapsed: 0:00:03.
  Batch   400  of  1,623.    Elapsed: 0:00:04.
  Batch   500  of  1,623.    Elapsed: 0:00:06.
  Batch   600  of  1,623.    Elapsed: 0:00:07.
  Batch   700  of  1,623.    Elapsed: 0:00:08.
  Batch   800  of  1,623.    Elapsed: 0:00:09.
  Batch   900  of  1,623.    Elapsed: 0:00:10.
  Batch 1,000  of  1,623.    Elapsed: 0:00:11.
  Batch 1,100  of  1,623.    Elapsed: 0:00:12.
  Batch 1,200  of  1,623.    Elapsed: 0:00:13.
  Batch 1,300  of  1,623.    Elapsed: 0:00:15.
  Batch 1,400  of  1,623.    Elapsed: 0:00:16.
  Batch 1,500  of  1,623.    Elapsed: 0:00:17.
  Batch 1,600  of  1,623.    Elapsed: 0:00:18.

Test took: 0:00:18


In [48]:
test_result['Predicted']

0           neutral
1          surprise
2             anger
3           neutral
4       non-neutral
           ...     
1618    non-neutral
1619            joy
1620        neutral
1621        neutral
1622            joy
Name: Predicted, Length: 1623, dtype: object

In [49]:
test_result

,id,utterance,Predicted
0,0,"Alright, whadyou do with him?",neutral
1,1,Oh! You're awake!,surprise
2,2,Then you gotta come clean with Ma! This is not...,anger
3,3,"Yeah, but this is",neutral
4,4,I don't wanna hear it! Now go to my room!,non-neutral
...,...,...,...
1618,1618,Nooo.,non-neutral
1619,1619,"Hi, Kate!",joy
1620,1620,"Hi, Lauren.",neutral
1621,1621,"Hi, Lauren.",neutral


In [50]:
test_result.drop(labels='utterance', axis="columns", inplace=True)

In [51]:
DATA_OUT_NAME = "Friends_electra3_3MaxLen:"+ str(MAX_LEN) + "_BchSize:"+ str(batch_size) + "_Epos"+ str(epochs) + ".csv"

test_csv = test_result.to_csv(DATA_OUT_NAME, columns=['id', 'Predicted'], index=False)
from google.colab import files

files.download(DATA_OUT_NAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


# f1-score parameter
from sklearn.metrics import f1_score
f1_score_avg = []
trues = []
preds = []

In [53]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

   #
    pred_flat = np.argmax(logits, axis=1).flatten()
    trues_flat = label_ids.flatten()
    trues.extend(trues_flat)
    preds.extend(pred_flat)
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  f1 score macro: {0: 2f}".format(f1_score(y_true=trues,y_pred= preds, labels = [0,1,2,3,4,5,6,7], average='macro')))
print("  f1 score micro: {0: 2f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1,2,3,4,5,6,7],average='micro')))
print("  f1 score weighted: {0: 2f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1,2,3,4,5,6,7],average='weighted')))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.60
  f1 score macro:  0.385977
  f1 score micro:  0.596237
  f1 score weighted:  0.584074
Test took: 0:00:01


In [54]:
test_acc = pd.DataFrame(
    {'trues':encoder.classes_[trues]
    ,'preds':encoder.classes_[preds]
    }
  )

In [55]:
test_acc

,trues,preds
0,neutral,surprise
1,neutral,neutral
2,neutral,neutral
3,neutral,neutral
4,neutral,neutral
...,...,...
2759,non-neutral,neutral
2760,non-neutral,anger
2761,non-neutral,neutral
2762,joy,joy


In [56]:
acc = (test_acc.trues == test_acc.preds).mean()

In [57]:
acc

0.5962373371924746

In [58]:
acc_idx = test_acc['preds'][test_acc['preds']==test_acc['trues']].index

In [59]:
test_acc.loc[acc_idx].groupby(['preds']).size().reset_index(name = 'count')

,preds,count
0,anger,44
1,disgust,8
2,joy,207
3,neutral,1012
4,non-neutral,184
5,sadness,26
6,surprise,167


In [60]:
test_acc.groupby(['trues']).size().reset_index(name = 'count')

,trues,count
0,anger,161
1,disgust,68
2,fear,32
3,joy,304
4,neutral,1287
5,non-neutral,541
6,sadness,85
7,surprise,286
